# First look into the King County Washington data

In [91]:
# import packages from Matt's Code Challege solution, this will be updated later
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from bs4 import BeautifulSoup
import requests
import json
%matplotlib inline

In [96]:
# Pulling in first data set
path = "../../../data/kc_house_data.csv"
df = pd.read_csv(path)
df.drop_duplicates(subset="id", inplace=True)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [97]:
# Looking for unique zipcodes from May 2014 - June 2014
zipcode=df["zipcode"].unique()
zipcode

array([98178, 98125, 98028, 98136, 98074, 98053, 98003, 98198, 98146,
       98038, 98007, 98115, 98107, 98126, 98019, 98103, 98002, 98133,
       98040, 98092, 98030, 98119, 98112, 98052, 98027, 98117, 98058,
       98001, 98056, 98166, 98023, 98070, 98148, 98105, 98042, 98008,
       98059, 98122, 98144, 98004, 98005, 98034, 98075, 98116, 98010,
       98118, 98199, 98032, 98045, 98102, 98077, 98108, 98168, 98177,
       98065, 98029, 98006, 98109, 98022, 98033, 98155, 98024, 98011,
       98031, 98106, 98072, 98188, 98014, 98055, 98039], dtype=int64)

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21420 non-null  int64  
 1   date           21420 non-null  object 
 2   price          21420 non-null  float64
 3   bedrooms       21420 non-null  int64  
 4   bathrooms      21420 non-null  float64
 5   sqft_living    21420 non-null  int64  
 6   sqft_lot       21420 non-null  int64  
 7   floors         21420 non-null  float64
 8   waterfront     19067 non-null  float64
 9   view           21357 non-null  float64
 10  condition      21420 non-null  int64  
 11  grade          21420 non-null  int64  
 12  sqft_above     21420 non-null  int64  
 13  sqft_basement  21420 non-null  object 
 14  yr_built       21420 non-null  int64  
 15  yr_renovated   17616 non-null  float64
 16  zipcode        21420 non-null  int64  
 17  lat            21420 non-null  float64
 18  long  

In [99]:
# DataFrame with only Price and Zipcode for merging
df_price=df[["zipcode","price"]]
df_price

,zipcode,price
0,98178,221900.0
1,98125,538000.0
2,98028,180000.0
3,98136,604000.0
4,98074,510000.0
...,...,...
21592,98103,360000.0
21593,98146,400000.0
21594,98144,402101.0
21595,98027,400000.0


In [146]:
url = 'https://www.publicschoolreview.com/washington/king-county'
response = requests.get(url,timeout=30)
soup = BeautifulSoup(response.content, 'html.parser')
list_school=soup.find(id="list_school")
school_items = list_school.find_all(class_="table_row row_click")
school_items.find(class_='rank-score-cnt rank-color-10').get_text()

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [141]:
school_items[0]

<li class="table_row row_click"><div class="school-type-list-text"><span class="table_cell_other_1 top_school"><span class="just-mobile-show-inline fw-600">Rank: #1</span><span class="rank"><span class="just-desktop-show-inline">1.</span></span><div class="school_info"><a class="school_links" href="/cascadia-elementary-school-profile/98115"><span class="tooltip" school_site="PSR" schoolidid="145838">Cascadia Elementary School</span></a> <span class="caption captionwidth"><strong>Magnet School</strong><br/><strong>Math: 98% | Reading: ≥99% <br/><div class="bubble_text" data-rel="tooltip" title="Denotes Washington state ranking (out of 10). A score of 10/10 indicates that student test scores for this school are in the top 1% among all WA schools.">Rank</div>: <div class="rank-score-cnt rank-color-10">10/<div class="total-rank-score">10</div></div>  Top 1%</strong></span><div class="compare_button"><label class="check-label"><input class="checkbox-input" id="compare_145838" type="checkbox

In [49]:
path_school = "../../../data/Public_Schools.csv"
df2 = pd.read_csv(path_school,index_col="OBJECTID")
print(df2.shape)
df2.head(20)

(107, 16)


,X,Y,TYPE,SCHOOL,ADDRESS,SE_ANNO_CAD_DATA,NAME,GRADE,CITY,ZIP,PHONE,WEBSITE,XCOORD,YCOORD,SITE_USE,PRJ_ENRLLMNT
OBJECTID,,,,,,,,,,,,,,,,
1,-122.353265,47.632023,NonStandard,QUEEN ANNE GYM,1431 2nd Ave N,NaN,Queen Anne Gym,9-12,Seattle,98109,NaN,NaN,1.265681e+06,234243.291153,Active,NaN
2,-122.315105,47.585469,Elementary,BEACON HILL ELEM,2025 14th Ave. S,NaN,Beacon Hill Int'l,K-5,Seattle,98144,(206) 252-2700,http://beaconhilles.seattleschools.org/,1.274761e+06,217081.839912,Active,NaN
3,-122.362141,47.548620,Option Elementary,BOREN BLDG,5950 Delridge Way SW,NaN,Louisa Boren STEM,K-8,Seattle,98106,(206)252-8450,http://borenstemk8.seattleschools.org/,1.262888e+06,203870.059905,Active,NaN
4,-122.324691,47.523611,Elementary,CONCORD ELEM,723 S Concord St.,NaN,Concord Int'l,K-5,Seattle,98108,(206) 252-8100,http://concordes.seattleschools.org/,1.271958e+06,194568.029953,Active,NaN
5,-122.313912,47.552372,Option High School,CLEVELAND HIGH SCHOOL,5511 15th Ave S,NaN,Cleveland STEM,9-12,Seattle,98108,(206)252-7800,http://clevelandhs.seattleschools.org/,1.274823e+06,205005.729878,Active,NaN
6,-122.294531,47.707629,High School,NATHAN HALE HIGH SCHOOL,10750 30th Ave NE,NaN,Nathan Hale,9-12,Seattle,98125,(206)252-3680,http://halehs.seattleschools.org/,1.280683e+06,261536.279874,Active,NaN
7,-122.374927,47.532838,Elementary,HUGHES,7740 34th Ave SW,NaN,Roxhill at E.C. Hughes,K-5,Seattle,98126,(206)252-9570,http://roxhilles.seattleschools.org/,1.259616e+06,198177.529952,Interim,NaN
8,-122.306755,47.723686,Elementary,OLYMPIC HILLS ELEM,13018 20th Ave NE,NaN,Olympic Hills,K-5,Seattle,98125,(206)413-2200,http://olympichillses.seattleschools.org/,1.277786e+06,267449.850068,Active,NaN
9,-122.303014,47.574045,Elementary,KIMBALL ELEM,3200 23rd Ave S,NaN,Kimball,K-5,Seattle,98144,(206)252-7280,http://kimballes.seattleschools.org/,1.277665e+06,212858.329986,Active,NaN


In [48]:
df2.columns

Index(['X', 'Y', 'TYPE', 'SCHOOL', 'ADDRESS', 'SE_ANNO_CAD_DATA', 'NAME',
       'GRADE', 'CITY', 'ZIP', 'PHONE', 'WEBSITE', 'XCOORD', 'YCOORD',
       'SITE_USE', 'PRJ_ENRLLMNT'],
      dtype='object')

In [50]:
# Removing columns that do not add value tot 
df2.drop(columns=['ADDRESS','SE_ANNO_CAD_DATA','CITY','PHONE','WEBSITE','PRJ_ENRLLMNT','XCOORD','YCOORD'],inplace=True)

In [59]:
df2_active=df2[df2["SITE_USE"]=="Active"].copy()

In [61]:
# 3 schools are listed as Interim I am interested in Active schools.
display(df2.shape,df2_active.shape)

(107, 8)

(104, 8)

In [79]:
zipcode_school=df2_active["ZIP"].unique()

In [72]:
df2_school=df2_active[["SCHOOL","ZIP"]]
df2_school.rename(columns={"ZIP":"zipcode"})

,SCHOOL,zipcode
OBJECTID,,
1,QUEEN ANNE GYM,98109
2,BEACON HILL ELEM,98144
3,BOREN BLDG,98106
4,CONCORD ELEM,98108
5,CLEVELAND HIGH SCHOOL,98108
...,...,...
114,MAGNOLIA,98199
115,WEST WOODLAND ELEM,98107
116,SOUTH LAKE HIGH SCHOOL,98118


In [147]:
# API testing for different data sets that might be useful

url = 'https://opendata.arcgis.com/datasets/bace62518e9e4b5fbd6dfc386380d417_416.geojson'
school_data = requests.get(url)

In [148]:
print(school_data)

<Response [200]>


In [149]:
district_api = school_data.json()
df_district_api=pd.DataFrame(district_api)
df_district_api.head()
# df_bulding_permit.drop(columns=[])

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.